In [3]:
import bs4
import chromadb 
from langchain import hub
from langchain_chroma import Chroma
from langchain_ollama import ChatOllama
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.prompts import ChatPromptTemplate

import os

In [4]:
llm_1b = ChatOllama(name="chat_llama3", model="llama3.2:1b",temperature=0)
llm_8b = ChatOllama(model="krith/meta-llama-3.1-8b-instruct:IQ2_M",temperature=0)

In [8]:
embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
)


In [6]:
documents = [
  "Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels",
  "Llamas were first domesticated and used as pack animals 4,000 to 5,000 years ago in the Peruvian highlands",
  "Llamas can grow as much as 6 feet tall though the average llama between 5 feet 6 inches and 5 feet 9 inches tall",
  "Llamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight",
  "Llamas are vegetarians and have very efficient digestive systems",
  "Llamas live to be about 20 years old, though some only live for 15 years and others live to be 30 years old",
]


In [10]:
vectorstore = InMemoryVectorStore.from_texts(
    documents,
    embedding=embeddings,
)

In [11]:
retriever = vectorstore.as_retriever()

In [12]:
retrieved_documents = retriever.invoke("How big is llamas ?")

In [13]:
retrieved_documents[0].page_content

'Llamas can grow as much as 6 feet tall though the average llama between 5 feet 6 inches and 5 feet 9 inches tall'

In [15]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def inspect(state):
    """Print the state passed between Runnables in a langchain and pass it on"""
    print(state)
    return state

prompt = """
You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question} 

Context: {context} 

Answer:
"""

rag_prompt = ChatPromptTemplate.from_messages({'system_message',prompt})

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | RunnableLambda(inspect)
    | rag_prompt
    | llm_8b
    | StrOutputParser()
)

rag_chain.invoke("How big is elephant?")

{'context': "Llamas can grow as much as 6 feet tall though the average llama between 5 feet 6 inches and 5 feet 9 inches tall\n\nLlamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight\n\nLlamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels\n\nLlamas are vegetarians and have very efficient digestive systems", 'question': 'How big is elephant?'}


"I don't know."

In [16]:
# Chroma db embeddings for persistent vector database

In [17]:
#db = Chroma(embedding_function=embeddings, persist_directory="/Users/insan.ramadhan/Documents/works/LLM/llama/llm_project/3.retriever/data/chroma_db")
db = Chroma.from_texts(documents, embeddings, persist_directory="/Users/insan.ramadhan/Documents/works/LLM/llama/llm_project/3.retriever/data/chroma_db2")

In [18]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})

In [19]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def inspect(state):
    """Print the state passed between Runnables in a langchain and pass it on"""
    print(state)
    return state

prompt = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question} 

Context: {context} 

Answer:
"""

rag_prompt = ChatPromptTemplate.from_messages({'system_message',prompt})

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | RunnableLambda(inspect)
    | rag_prompt
    | llm_8b
    | StrOutputParser()
)

rag_chain.invoke("How big is elephant?")

{'context': 'Llamas can grow as much as 6 feet tall though the average llama between 5 feet 6 inches and 5 feet 9 inches tall\n\nLlamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight', 'question': 'How big is elephant?'}


"I don't have enough information about elephants in the provided context. However, I can tell you that there is no mention of elephant size or weight in the given text, which only talks about llamas."